In [ ]:
'''
Author: Mingxin Zhang m.zhang@hapis.u-tokyo.ac.jp
Date: 2023-03-06 03:29:29
LastEditors: Mingxin Zhang
LastEditTime: 2023-04-12 01:54:25
Copyright (c) 2023 by Mingxin Zhang, All Rights Reserved. 
'''

'\nAuthor: Mingxin Zhang m.zhang@hapis.u-tokyo.ac.jp\nDate: 2023-03-06 03:29:29\nLastEditors: Mingxin Zhang\nLastEditTime: 2023-04-12 01:54:25\nCopyright (c) 2023 by Mingxin Zhang, All Rights Reserved. \n'

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import pickle
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn import preprocessing
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
class Generator(nn.Module):
    def __init__(self, encoded_space_dim):
        super().__init__()
        self.fc1 = nn.Linear(encoded_space_dim, 32 * 1 * 23)
        self.unflatten = nn.Unflatten(dim=1, unflattened_size=(32, 1, 23))

        self.deconv1 = nn.ConvTranspose2d(32, 16, 5, stride=2, output_padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.deconv2 = nn.ConvTranspose2d(16, 8, 5, stride=2, padding=2, output_padding=1)
        self.bn2 = nn.BatchNorm2d(8)
        self.deconv3 = nn.ConvTranspose2d(8, 1, 3, padding=1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.unflatten(x)
        x = F.relu(self.bn1(self.deconv1(x)))
        x = F.relu(self.bn2(self.deconv2(x)))
        x = self.deconv3(x)
        return x


class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        ### Convolutional section
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.conv2 = nn.Conv2d(8, 16, 5, padding=2)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 5, padding=0)
        self.bn2 = nn.BatchNorm2d(32)

        self.pool = nn.MaxPool2d(2)

        self.flatten = nn.Flatten(start_dim=1)

        self.fc1 = nn.Linear(32 * 1 * 23, 1)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(self.conv2(x))
        x = F.relu(self.bn1(x))
        x = self.pool(self.conv3(x))
        x = F.relu(self.bn2(x))
        x = self.flatten(x)
        x = F.sigmoid(self.fc1(x))
        return x

In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

with open('/content/drive/MyDrive/Colab Notebooks/vibrotactile-encoder/trainset.pickle', 'rb') as file:
# with open('trainset.pickle', 'rb') as file:
    trainset = pickle.load(file)

spectrogram = torch.from_numpy(trainset['spectrogram'].astype(np.float32))
texture = trainset['texture']
le = preprocessing.LabelEncoder()
labels = torch.as_tensor(le.fit_transform(texture))

train_dataset = torch.utils.data.TensorDataset(spectrogram, labels) 
train_dataloader = torch.utils.data.DataLoader( 
    dataset = train_dataset, 
    batch_size = 64,
    shuffle = True, 
    num_workers = 2, 
    ) 

adversarial_loss = nn.BCELoss()

FEAT_DIM = 16
generator= Generator(encoded_space_dim = FEAT_DIM)
discriminator = Discriminator()

gen_lr = 5e-4
dsc_lr = 5e-4

optimizer_G = optim.Adam(generator.parameters(), lr=gen_lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=dsc_lr)

generator.to(device)
discriminator.to(device)

epoch_num = 150

Selected device: cuda


In [6]:
EPS = 1e-15

for epoch in range(1, epoch_num + 1):
    generator.train()
    discriminator.train()

    for i, (img, label) in enumerate(train_dataloader):
        img = torch.unsqueeze(img, 1) # Add channel axis (1 channel)
        img = img.to(device)

        valid = torch.autograd.Variable(torch.Tensor(img.size(0), 1).fill_(1.0), requires_grad=False).to(device)
        fake = torch.autograd.Variable(torch.Tensor(img.size(0), 1).fill_(0.0), requires_grad=False).to(device)

        # 1) generator
        optimizer_G.zero_grad()
        # input noise
        z = torch.autograd.Variable(torch.Tensor(np.random.normal(0, 1, (img.shape[0], FEAT_DIM)))).to(device)
        # train generator
        gen_img = generator(z)
        g_loss = adversarial_loss(discriminator(gen_img), valid)
        g_loss.backward()
        optimizer_G.step()

        # 2) discriminator
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(img), valid)
        fake_loss = adversarial_loss(discriminator(gen_img.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

    print('Epoch: ', epoch, 'Generator Loss: ', g_loss.item(), 'Discriminator Loss: ', d_loss.item())

torch.save(generator.state_dict(), '/content/drive/MyDrive/Colab Notebooks/vibrotactile-encoder/generator_' + str(FEAT_DIM) + 'd.pt')
torch.save(discriminator.state_dict(), '/content/drive/MyDrive/Colab Notebooks/vibrotactile-encoder/discriminator_' + str(FEAT_DIM) + 'd.pt')

Epoch:  1 Generator Loss:  5.76810359954834 Discriminator Loss:  0.0034423479810357094
Epoch:  2 Generator Loss:  4.635915279388428 Discriminator Loss:  0.016885820776224136
Epoch:  3 Generator Loss:  4.107100486755371 Discriminator Loss:  0.015812300145626068
Epoch:  4 Generator Loss:  5.162808418273926 Discriminator Loss:  0.018998153507709503
Epoch:  5 Generator Loss:  3.7049834728240967 Discriminator Loss:  0.04880311340093613
Epoch:  6 Generator Loss:  4.147192478179932 Discriminator Loss:  0.046294040977954865
Epoch:  7 Generator Loss:  3.623947858810425 Discriminator Loss:  0.04935571178793907
Epoch:  8 Generator Loss:  4.674008369445801 Discriminator Loss:  0.0529303252696991
Epoch:  9 Generator Loss:  4.990487575531006 Discriminator Loss:  0.057409100234508514
Epoch:  10 Generator Loss:  3.858515977859497 Discriminator Loss:  0.06523606926202774
Epoch:  11 Generator Loss:  4.171542167663574 Discriminator Loss:  0.09135007858276367
Epoch:  12 Generator Loss:  4.165359973907471 

In [7]:
generator = Generator(encoded_space_dim = FEAT_DIM)
discriminator = Discriminator()

generator_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/vibrotactile-encoder/generator_' + str(FEAT_DIM) + 'd.pt', map_location=torch.device('cpu'))
generator_dict = {k: v for k, v in generator_dict.items()}

discriminator_dict = torch.load('/content/drive/MyDrive/Colab Notebooks/vibrotactile-encoder/discriminator_' + str(FEAT_DIM) + 'd.pt', map_location=torch.device('cpu'))
discriminator_dict = {k: v for k, v in discriminator_dict.items()}

generator.load_state_dict(generator_dict)
discriminator.load_state_dict(discriminator_dict)

RuntimeError: ignored